In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import SimpleITK as sitk
import tensorflow as tf
from tensorflow import keras
from keras.layers import AveragePooling3D
import time
import copy
from sklearn.cluster import KMeans
import pandas as pd
import gc
from sklearn.linear_model import LinearRegression
import math
import tensorflow as tf
from multiprocessing import Process, Manager
import datetime
import sys
import csv
import os
from tensorflow.keras.utils import to_categorical
from data_generate import *  # generate parameters and simulate data
from KEM_SIMU import *  # KEM class
from simulation_auxiliary import *  # bandwidth selection function
sys.path.append('../')
from utils import *

%matplotlib inline

In [2]:
K = 3
# 数据的路径
lung_image_path = "/database/datasets/Classics/LUNA2016/IMAGES/"  # 存放CT的文件夹
lung_mask_path = "/database/datasets/Classics/LUNA16-Mask/"  # 存放肺部mask的文件夹
# the absolute paths of the CT files
lung_image_file_list = glob(lung_image_path + "*.mhd")

stats_dict = {'lung_image_path': lung_image_path,
              'lung_mask_path': lung_mask_path,
              'lung_image_file_list': lung_image_file_list,
              }

# Bandwidth Selection

- choose kernel sizes and bandwidth candidates for CV and REG methods

- coefficients for CV: [0.5, 0.9, 1.3, 1.7, 2.1]

In [3]:
kernel_size_list = np.arange(3, 12, 2)

# CV: kernel size list and bandwidth candidate list
CV_kernel_size_list = []
CV_bandwidth_list = []
for kernel_size in kernel_size_list:
    for coef in np.arange(start=0.5, stop=2.5, step=2/5):
        CV_kernel_size_list.append(kernel_size)
        CV_bandwidth_list.append(kernel_size / 512 * coef)

# REG: kernel size list and bandwidth candidate list
REG_kernel_size_list = []
REG_bandwidth_list = []
for kernel_size in kernel_size_list:
    for coef in [1]:
        REG_kernel_size_list.append(kernel_size)
        REG_bandwidth_list.append(kernel_size / 512 * coef)

print(f"length of CV candidates: {len(CV_kernel_size_list)}")
print(f"length of REG candidates: {len(REG_kernel_size_list)}")

# save to stats_dict
stats_dict['CV_kernel_size_list'] = CV_kernel_size_list
stats_dict['CV_bandwidth_list'] = CV_bandwidth_list
stats_dict['REG_kernel_size_list'] = REG_kernel_size_list
stats_dict['REG_bandwidth_list'] = REG_bandwidth_list

length of CV candidates: 25
length of REG candidates: 5


In [4]:
Ch_path = f"./Ch-{datetime.date.today().isoformat()}.csv"

with open(Ch_path, 'w', newline = '', encoding='utf-8') as f:
    csv_write = csv.writer(f)  
    csv_write.writerow([f"patient", "data_gen_time", "Ch_CV", "time_CV", "Ch_REG", "time_REG", "model.coef0", "model.coef1"])

In [5]:
# randomly select 100 patients for bandwidth selection
np.random.seed(0)
index_list = np.arange(0, len(lung_image_file_list), 1)
np.random.shuffle(index_list)
index_list = index_list[0:100]

# transform the dict so that it can be used in the following Process
manager = Manager()
stats_dict = manager.dict(stats_dict)

In [6]:
for index in index_list:
    np.random.seed(index)
    tf.random.set_seed(index)
    stats_dict['index'] = index
    process_eval = Process(target=compute_Ch_4_cv_reg, 
                           args=(stats_dict, Ch_path, 0.8))
    process_eval.start()
    process_eval.join()

---------------WE ARE LOADING 150th PATIENT's CT with shape (292, 512, 512)...---------------


2023-10-30 03:51:32.661475: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 03:51:34.540041: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 6.0
2023-10-30 03:51:34.544238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15401 MB memory:  -> device: 1, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b4:00.0, compute capability: 6.0
2023-10-30 03:51:34.548038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/de

---------------WE ARE LOADING 406th PATIENT's CT with shape (147, 512, 512)...---------------


2023-10-30 03:51:47.444614: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 03:51:49.325765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 6.0
2023-10-30 03:51:49.328001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15401 MB memory:  -> device: 1, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b4:00.0, compute capability: 6.0
2023-10-30 03:51:49.329643: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/de

---------------WE ARE LOADING 513th PATIENT's CT with shape (110, 512, 512)...---------------


2023-10-30 03:52:01.692749: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 03:52:03.591151: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 6.0
2023-10-30 03:52:03.593430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15401 MB memory:  -> device: 1, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b4:00.0, compute capability: 6.0
2023-10-30 03:52:03.595154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/de

KeyboardInterrupt: 

## Re-Run the CT with Large Depth by Better GPU

In [7]:
# for i, index in enumerate(index_list):
#     # set random seed
#     np.random.seed(index)
#     tf.random.set_seed(index)
#     # read data and record its depth
#     stats_dict['index'] = index
#     sample_CT_path = stats_dict['lung_image_file_list'][index]
#     sample_CT_array, lung_mask_array = get_original_data_newversion(stats_dict['lung_image_path'], stats_dict['lung_mask_path'], sample_CT_path)
#     depth = sample_CT_array.shape[0]
#     info_list = [f"patient_{index}", depth]
#     # record results
#     with open("./optimal bwd csv/random100_all.csv", 'a', newline = '', encoding='utf-8') as f:
#         csv_write = csv.writer(f)  
#         csv_write.writerow(info_list)

In [8]:
# CT index and depth
random100 = pd.read_csv("./optimal bwd csv/random100_all.csv")
# results at hand
Ch_df = pd.read_csv(Ch_path)
# merge results; if NaN exists, then we need to re-run this row
random100_all = pd.merge(random100, Ch_df, how="left", on="patient")
random100_all

,patient,depth,data_gen_time,Ch_CV,time_CV,Ch_REG,time_REG,model.coef0,model.coef1
0,patient_150,292,15.425414,0.139164,376.840242,0.238385,184.864365,0.071734,1.046064e-06
1,patient_406,147,7.923266,0.227100,260.861032,0.226362,90.681398,0.181767,1.845100e-06
2,patient_513,110,5.991382,0.217888,187.628968,0.214569,65.960526,0.142470,9.944276e-07
3,patient_101,197,9.840834,0.131556,334.943539,0.233110,117.309187,0.150715,1.879123e-06
4,patient_584,383,20.560244,0.144662,504.758250,0.250721,243.679065,0.182814,3.795135e-06
...,...,...,...,...,...,...,...,...,...
95,patient_382,127,7.421021,0.222407,224.812824,0.220602,77.938102,0.214699,1.819602e-06
96,patient_687,115,6.586531,0.219274,204.277602,0.217178,71.249134,0.251859,1.913139e-06
97,patient_475,116,6.816620,0.219546,202.712783,0.215272,71.737118,0.193411,1.381245e-06
98,patient_204,140,7.977019,0.225523,241.112269,0.220903,83.966350,0.287176,2.457205e-06


In [9]:
rerun_df = random100_all.loc[np.isnan(random100_all.Ch_CV), ]
rerun_index_list = [int(item.split('_')[1]) for item in rerun_df.patient]

In [ ]:
for index in rerun_index_list:
    np.random.seed(index)
    tf.random.set_seed(index)
    stats_dict['index'] = index
    process_eval = Process(target=compute_Ch_4_cv_reg, 
                           args=(stats_dict, CT_path, 0.8))
    process_eval.start()
    process_eval.join()

## Estimation Accuracy Compare under Optimal Bandwidth

In [10]:
bdw_mse_path = f"./bdw_mse_df-{datetime.date.today().isoformat()}.csv"

with open(bdw_mse_path, 'w', newline = '', encoding='utf-8') as f:
    csv_write = csv.writer(f)  
    csv_write.writerow(["patient", "CV_pi_rmse", "CV_mu_rmse", "CV_sigma_rmse", 
                                   "REG_pi_rmse", "REG_mu_rmse", "REG_sigma_rmse"])

In [11]:
# CT index and depth
random100 = pd.read_csv("./optimal bwd csv/random100_all.csv")
# results at hand
Ch_df = pd.read_csv(Ch_path)
# merge results; no NaN exists
random100_all = pd.merge(random100, Ch_df, how="left", on="patient")

# compute optimal bandwidth
Ch_df = random100_all.dropna()
Ch_df['sample_size'] = (Ch_df['depth'] * 512 * 512 * 0.8).map(int)
Ch_df['bdw_CV'] = Ch_df['Ch_CV'] * Ch_df['sample_size'].map(lambda x: x**(-1/7))
Ch_df['bdw_REG'] = Ch_df['Ch_REG'] * Ch_df['sample_size'].map(lambda x: x**(-1/7))
Ch_df['index'] = [int(item.split('_')[1]) for item in list(Ch_df.patient)]
Ch_df.head()

,patient,depth,data_gen_time,Ch_CV,time_CV,Ch_REG,time_REG,model.coef0,model.coef1,sample_size,bdw_CV,bdw_REG,index
0,patient_150,292,15.425414,0.139164,376.840242,0.238385,184.864365,0.071734,1.046064e-06,61236838,0.010742,0.018401,150
1,patient_406,147,7.923266,0.227100,260.861032,0.226362,90.681398,0.181767,1.845100e-06,30828134,0.019336,0.019273,406
2,patient_513,110,5.991382,0.217888,187.628968,0.214569,65.960526,0.142470,9.944276e-07,23068672,0.019336,0.019042,513
3,patient_101,197,9.840834,0.131556,334.943539,0.233110,117.309187,0.150715,1.879123e-06,41313894,0.010742,0.019035,101
4,patient_584,383,20.560244,0.144662,504.758250,0.250721,243.679065,0.182814,3.795135e-06,80320921,0.010742,0.018618,584


In [12]:
bandwidth_mse_df = pd.read_csv(bdw_mse_path)

if bandwidth_mse_df.shape[0] > 0:  # run it the first time
    tmp_df = pd.merge(Ch_df, bandwidth_mse_df, how='left', on="patient")
    tmp_df = tmp_df.loc[np.isnan(tmp_df.CV_pi_rmse),]
else:  # rerun for large-depth CT data
    tmp_df = Ch_df
stats_dict['tmp_df'] = tmp_df
tmp_df

,patient,depth,data_gen_time,Ch_CV,time_CV,Ch_REG,time_REG,model.coef0,model.coef1,sample_size,bdw_CV,bdw_REG,index,CV_pi_rmse,CV_mu_rmse,CV_sigma_rmse,REG_pi_rmse,REG_mu_rmse,REG_sigma_rmse


- may need to rerun with a better GPU for large-depth CT data

In [13]:
for i in range(len(tmp_df)):
    # 随机数种子
    index = tmp_df.iloc[i, ]["index"]
    np.random.seed(index)
    tf.random.set_seed(index)
    stats_dict['index'] = index
    stats_dict['i'] = i    
    process_eval = Process(target=compute_rmse_4_cv_reg, 
                           args=(stats_dict, bdw_mse_path))
    process_eval.start()
    process_eval.join()
    if process_eval.exitcode == 1:
        print(f"Failed with index:{index}")

In [14]:
# optimal bandwidth
Ch_df = pd.read_csv(Ch_path)
Ch_df = pd.merge(random100, Ch_df, how="left", on="patient")
Ch_df = Ch_df.dropna()

# rmse results based on the optimal bandwidth
bdw_mse_df = pd.read_csv(bdw_mse_path)

# merge the two dfs for complete data
complete_data = pd.merge(Ch_df, bdw_mse_df, how="left", on="patient")
# complete_data.to_csv('./complete_Ch_bwd_mse-2023-01-13.csv')
complete_data

,patient,depth,data_gen_time,Ch_CV,time_CV,Ch_REG,time_REG,model.coef0,model.coef1,CV_pi_rmse,CV_mu_rmse,CV_sigma_rmse,REG_pi_rmse,REG_mu_rmse,REG_sigma_rmse
0,patient_150,292,15.425414,0.139164,376.840242,0.238385,184.864365,0.071734,1.046064e-06,0.042868,0.009482,0.005703,0.028446,0.008415,0.004022
1,patient_406,147,7.923266,0.227100,260.861032,0.226362,90.681398,0.181767,1.845100e-06,0.023817,0.007346,0.003916,0.023824,0.007366,0.003915
2,patient_513,110,5.991382,0.217888,187.628968,0.214569,65.960526,0.142470,9.944276e-07,0.027350,0.009342,0.005849,0.027372,0.009303,0.005833
3,patient_101,197,9.840834,0.131556,334.943539,0.233110,117.309187,0.150715,1.879123e-06,0.043812,0.014145,0.008411,0.028630,0.010465,0.004811
4,patient_584,383,20.560244,0.144662,504.758250,0.250721,243.679065,0.182814,3.795135e-06,0.041558,0.011599,0.007792,0.025331,0.006615,0.004038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,patient_382,127,7.421021,0.222407,224.812824,0.220602,77.938102,0.214699,1.819602e-06,0.025684,0.008516,0.004347,0.025697,0.008533,0.004345
96,patient_687,115,6.586531,0.219274,204.277602,0.217178,71.249134,0.251859,1.913139e-06,0.025235,0.009127,0.004566,0.025290,0.009177,0.004563
97,patient_475,116,6.816620,0.219546,202.712783,0.215272,71.737118,0.193411,1.381245e-06,0.026651,0.008092,0.005679,0.026720,0.008080,0.005695
98,patient_204,140,7.977019,0.225523,241.112269,0.220903,83.966350,0.287176,2.457205e-06,0.027840,0.010004,0.007842,0.027854,0.010036,0.007844


# Consistency实验

In [6]:
training_ratio_list = [0.10, 0.20, 0.50, 0.75, 1.00]
Ch = 0.2293052

manager = Manager()
stats_dict = manager.dict(stats_dict)

In [7]:
mse_path = f"./mse_results_DF-{datetime.date.today().isoformat()}.csv"

# if os.path.exists(mse_path) is True:
#     os.remove(mse_path)
with open(mse_path, 'w', newline = '', encoding='utf-8') as f:
    csv_write = csv.writer(f)  
    csv_write.writerow(["patient", "depth", "training_ratio", "kernel_size", "pi_rmse", "mu_rmse", "sigma_rmse"])

In [8]:
for index in range(len(lung_image_file_list)):
    # set random seed
    np.random.seed(index)
    tf.random.set_seed(index)
    stats_dict['index'] = index    
    process_eval = Process(target=compute_rmse_4_consistency, 
                           args=(stats_dict, mse_path,
                                 training_ratio_list, Ch,))
    process_eval.start()
    process_eval.join()

2023-10-30 07:45:30.653168: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 07:45:33.434736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15401 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b3:00.0, compute capability: 6.0
2023-10-30 07:45:33.436808: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15401 MB memory:  -> device: 1, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b4:00.0, compute capability: 6.0
2023-10-30 07:45:33.438632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:

------------- Ratio 0.1 -------------
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.0998 positions for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 1.293 seconds
	centers: [1.0004517  0.6165712  0.06453138]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 201, 512, 512, 1)
	pi_estimate: (3, 201, 512, 512, 1)
	mu_estimate: (3, 201, 512, 512, 1)
	sigma_estimate: (3, 201, 512, 512, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (11, 11, 11, 1, 1)
From function(kem_algorithm): Receive max_steps: 50.
########################## STEP 0 ##########################
+++ From m_step: add smooth_parameter to pik_estimate
	 Current pik difference: 0.222095
From function(kem_algorithm): E step success.
	 Current pi difference: 0.0283957
	 Current mu difference: 0.0010458
	 Current sigma difference: 0.00106291
From function(kem_algorithm): M step success.
From function(ke

2023-10-30 07:48:47.748240: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 07:48:50.643376: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15401 MB memory:  -> device: 0, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b3:00.0, compute capability: 6.0
2023-10-30 07:48:50.645046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 193 MB memory:  -> device: 1, name: Tesla P100-SXM2-16GB, pci bus id: 0000:b4:00.0, compute capability: 6.0
2023-10-30 07:48:50.646887: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/

------------- Ratio 0.1 -------------
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.09997 positions for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 2.442 seconds
	centers: [1.0015941  0.45175412 0.04741291]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 375, 512, 512, 1)
	pi_estimate: (3, 375, 512, 512, 1)
	mu_estimate: (3, 375, 512, 512, 1)
	sigma_estimate: (3, 375, 512, 512, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (11, 11, 11, 1, 1)
From function(kem_algorithm): Receive max_steps: 50.
########################## STEP 0 ##########################


Process Process-4:
Traceback (most recent call last):
  File "/root/miniconda3/envs/myconda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/myconda/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/KEM/KEM_simulation/simulation_auxiliary.py", line 252, in compute_rmse_4_consistency
    kem.kem_algorithm(max_steps=50, epsilon=1e-4, smooth_parameter=1e-20)
  File "/mnt/KEM/KEM_simulation/KEM_SIMU.py", line 93, in kem_algorithm
    self.e_step(smooth_parameter)
  File "/mnt/KEM/KEM_simulation/KEM_SIMU.py", line 114, in e_step
    pik_estimate = normal_density_function_tf((self.training_data - self.mu_estimate) / self.sigma_estimate) * (
  File "/mnt/KEM/KEM_simulation/../utils.py", line 91, in normal_density_function_tf
    return 1 / tf.sqrt(2 * np.pi) * tf.exp(-x ** 2 / 2)
  File "/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tensorflow/python/

------------- Ratio 0.1 -------------


KeyboardInterrupt: 